In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
 
# Configure depth and rgb and infrared streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
# config.enable_stream(rs.stream.infrared, 1024, 768, rs.format.y8, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipeline.start(config)



In [7]:
# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)

# Streaming loop
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 153
        depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

        # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((bg_removed, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()


Depth Scale is:  0.0002500000118743628


In [4]:
alignIt = rs.align(rs.stream.color) # rs.stream.depth
aligned_frame = alignIt.process(frame)
# i prefer frame = alignIt.process(frame)

In [ ]:
#!/usr/bin/env python3
import pyrealsense2 as rs
import numpy as np
import cv2
# import rospy
# from sensor_msgs.msg import Image
# from cv_bridge import CvBridge, CvBridgeError
# import message_filters

# import tf
# import tf.transformations as transformations
# from geometry_msgs.msg import PoseStamped  # 导入PoseStamped消息类型

from ultralytics import YOLO
import time
import openvino as ov
import torch
# import time

import deep_sort.deep_sort.deep_sort as ds
# import tf.transformations
# from FSM import State

# bridge = CvBridge()
#内参
fx = 897.6827392578125
fy = 898.0628051757812
cx = 649.2904052734375
cy = 376.3505859375
y_mouse,x_mouse = 100,100

# parent_dir = "/home/wu/Lab/yolov8-deepsort-fast/"
tracker = ds.DeepSort("/home/wu/catkin_ws/src/my_pkg/cv_pkg/scripts/ckpt.t8")
Rot = np.array([[0,0,1],[1,0,0],[0,-1,0]])
# tran = np.array([0,0,0.521])
# def mouse_callback(event, x, y, flags, param):
#     global x_mouse, y_mouse
#     if event == cv2.EVENT_LBUTTONDOWN:
#         x_mouse = x
#         y_mouse = y
#         print(f"Mouse clicked at: x={x}, y={y}")

def extract_detections(results, detect_class):
    """
    从模型结果中提取和处理检测信息。
    - results: YoloV8模型预测结果，包含检测到的物体的位置、类别和置信度等信息。
    - detect_class: 需要提取的目标类别的索引。
    参考: https://docs.ultralytics.com/modes/predict/#working-with-results
    """
    
    # 初始化一个空的二维numpy数组，用于存放检测到的目标的位置信息
    # 如果视频中没有需要提取的目标类别，如果不初始化，会导致tracker报错
    detections = np.empty((0, 4)) 
    
    confarray = [] # 初始化一个空列表，用于存放检测到的目标的置信度。
    
    # 遍历检测结果
    # 参考：https://docs.ultralytics.com/modes/predict/#working-with-results
    for r in results:
        for box in r.boxes:
            # 如果检测到的目标类别与指定的目标类别相匹配，提取目标的位置信息和置信度
            if box.cls[0].int() == detect_class:
                x1, y1, x2, y2 = box.xywh[0].int().tolist() # 提取目标的位置信息，并从tensor转换为整数列表。
                conf = round(box.conf[0].item(), 2) # 提取目标的置信度，从tensor中取出浮点数结果，并四舍五入到小数点后两位。
                detections = np.vstack((detections, np.array([x1, y1, x2, y2]))) # 将目标的位置信息添加到detections数组中。
                confarray.append(conf) # 将目标的置信度添加到confarray列表中。
    return detections, confarray # 返回提取出的位置信息和置信度。

def image_callback(color_image,aligned_depth_image):
    global x_mouse, y_mouse
    # current_state = rospy.get_param("current_state", "WAITING")  # 默认值为 "WAITING"

    # 检查状态是否为 TRACKING
    # if current_state != State.TRACKING:
    #     return  # 如果不是 TRACKING，则直接返回，不处理图像
    # try:
    #     cv_color_image = bridge.imgmsg_to_cv2(color_image, "bgr8")
    #     # print(depth_image)
    #     # cv_depth_image = bridge.imgmsg_to_cv2(depth_image, "32FC1")
    #     cv_aligned_depth_image = bridge.imgmsg_to_cv2(aligned_depth_image, "32FC1")
    # except CvBridgeError as e:
    #     print(e)
    #     return
    cv_color_image = color_image
    cv_aligned_depth_image = aligned_depth_image
    results = model.predict(cv_color_image,imgsz=480,half=True)
    detections, confarray = extract_detections(results, 0)
    resultsTracker = tracker.update(detections, confarray, cv_color_image)
    for x1, y1, x2, y2, Id in resultsTracker:
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Convert position to integers.  
        # print(f"Id: {Id}, x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")
        if Id == 1:
            x_mouse = (x1 + x2) // 2
            y_mouse = (y1 + y2) // 2
            print("detedted")
            
        cv2.rectangle(cv_color_image, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.putText(cv_color_image, "LID-" + str(int(Id)), (max(-10, x1), max(40, y1)), fontScale=1, fontFace=cv2.FONT_HERSHEY_SIMPLEX, color=(255, 255, 255), thickness=2)
    
    
    
    
    
    if 0 <= x_mouse < cv_aligned_depth_image.shape[1] and 0 <= y_mouse < cv_aligned_depth_image.shape[0]:
        depth = cv_aligned_depth_image[y_mouse, x_mouse] / 1000.0
        cam_x = (x_mouse - cx) * depth / fx
        cam_y = (y_mouse - cy) * depth / fy
        cam_z = depth
        # print("Aligned Depth at center pixel: ", depth)
        x,y,z = np.dot(Rot, np.array([cam_x,cam_y,cam_z]))
        yaw = np.arctan2(y,x)
        # quaternion = tf.transformations.quaternion_from_euler(0, 0, yaw)
        print(x,y,z)
        cv2.circle(cv_color_image, (x_mouse, y_mouse), 5, (0, 0, 255), -1)
        cv2.putText(cv_color_image, f"x={x:.2f}, y={y:.2f}, z={z:.2f}", (x_mouse, y_mouse), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("cv_color_image", cv_color_image)
        cv2.waitKey(1)    
    # 创建 PoseStamped 消息并发布
        
           

    
    


if __name__ == '__main__':
    # rospy.init_node('get_image', anonymous=True)
    import pyrealsense2 as rs
    import numpy as np
    import cv2
    import os
    
    # Configure depth and rgb and infrared streams
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
    # config.enable_stream(rs.stream.infrared, 1024, 768, rs.format.y8, 30)
    config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
    profile = pipeline.start(config)
    # Getting the depth sensor's depth scale (see rs-align example for explanation)
    depth_sensor = profile.get_device().first_depth_sensor()
    depth_scale = depth_sensor.get_depth_scale()
    print("Depth Scale is: " , depth_scale)

    # We will be removing the background of objects more than
    #  clipping_distance_in_meters meters away
    clipping_distance_in_meters = 1 #1 meter
    clipping_distance = clipping_distance_in_meters / depth_scale

    # 创建订阅对象
    # color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
    # # depth_sub = message_filters.Subscriber('/camera/depth/image_rect_raw', Image)
    # align_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
    
    # pose_pub = rospy.Publisher('goal_pose', PoseStamped, queue_size=1)

    # 创建模型对象
    model = YOLO("/home/wu/catkin_ws/src/my_pkg/cv_pkg/scripts/yolov8n_ncnn_model")
    align_to = rs.stream.color
    align = rs.align(align_to)

# Streaming loop
    try:
        while True:
            # Get frameset of color and depth
            frames = pipeline.wait_for_frames()
            # frames.get_depth_frame() is a 640x360 depth image

            # Align the depth frame to color frame
            aligned_frames = align.process(frames)

            # Get aligned frames
            aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()

            # Validate that both frames are valid
            if not aligned_depth_frame or not color_frame:
                continue

            depth_image = np.asanyarray(aligned_depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            image_callback(color_image,depth_image)
            # # Remove background - Set pixels further than clipping_distance to grey
            # grey_color = 153
            # depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
            # bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

            # # Render images
            # depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            # images = np.hstack((bg_removed, depth_colormap))
            # cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
            # cv2.imshow('Align Example', images)
            # key = cv2.waitKey(1)
            # # Press esc or 'q' to close the image window
            # if key & 0xFF == ord('q') or key == 27:
            #     cv2.destroyAllWindows()
            #     break
    finally:
        pipeline.stop()
    # 创建时间同步对象
    # ts = message_filters.TimeSynchronizer([color_sub,align_sub], 10)  # 10是缓冲区大小
    # ts.registerCallback(image_callback)

    # rospy.spin()


Depth Scale is:  0.0002500000118743628
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /home/wu/catkin_ws/src/my_pkg/cv_pkg/scripts/yolov8n_ncnn_model for NCNN inference...

0: 480x480 (no detections), 73.8ms
Speed: 2.9ms preprocess, 73.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 480)
7.794 -4.76913416229966 2.3983583936262547

0: 480x480 (no detections), 80.3ms
Speed: 2.2ms preprocess, 80.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 480)
7.834 -4.793610088203173 2.4106671357028584

0: 480x480 (no detections), 72.5ms
Speed: 2.3ms preprocess, 72.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 480)
7.718 -4.722629903082984 2.3749717836807074

0: 480x480 (no detections), 79.8ms
Speed: 2.5ms preprocess, 79.8ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 480)
7.743 -4.737927356772681 2.38

KeyboardInterrupt: 

: 